In [3]:
import pandas as pd
from pathlib import Path

BASE = Path("..")  # parent folder: Understanding User Behavior

logs = pd.read_csv(BASE / "logs.csv")
sessions = pd.read_csv(BASE / "sessions.csv")
questionnaires = pd.read_csv(BASE / "questionnaires.csv")

In [ ]:
# Merge questionnaires (per user) with sessions (per session) to add interface_version
# We keep one row per user in this simple version.

sessions_unique = sessions[["user_id", "interface_version"]].drop_duplicates()

df_q = questionnaires.merge(
    sessions_unique,
    on="user_id",
    how="left"
)

df_q.head()

In [4]:
# Overall descriptive statistics
psych_cols = ["SUS_score", "NASA_TLX", "UES_engagement",
              "IMI_autonomy", "IMI_competence", "intention_reuse"]

df_q[psych_cols].describe()  # mean, std, min, quartiles, max

# Grouped by interface version
df_q.groupby("interface_version")[psych_cols].agg(["mean", "std", "min", "max"])

NameError: name 'df_q' is not defined